In [5]:
!pip install opencv-python opencv-contrib-python numpy scipy rarfile


# Importing database folder from Google Drive

In [23]:
from google.colab import drive
drive.mount('/content/drive')
import rarfile
import os

rar_path = '/content/drive/MyDrive/aDatabase/s1.rar'
extract_path = '/content/images'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
with rarfile.RarFile(rar_path) as rf:
    rf.extractall(extract_path)

# Load images from the database folder

In [25]:
import cv2
import numpy as np
import os
# Function to load images from a folder
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        if filename.endswith(".pgm"):  # Check if the file is a .pgm image
            img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE) # Read image in grayscale
            if img is not None:  # If the image is successfully loaded
                images[filename] = img
    return images


In [28]:
# Load images from the extracted folder
image_folder = extract_path
images = load_images_from_folder(image_folder)

# Face recognising LBPH algorithm:
LBPH is a popular algorithm for face recognition because it is simple and efficient. It works by:

Dividing the face image into several small regions.
Computing Local Binary Patterns (LBP) histograms for each region.
Concatenating these histograms into a single feature vector.
Comparing the feature vectors of different faces using a distance metric to determine similarity.


In [26]:
# Initialize the face recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

# Prepare training data
labels = []
training_images = []
label_dict = {}
# Loop over the images and prepare the data
for i, (filename, image) in enumerate(images.items()):
    labels.append(i)  # Append a label (index)
    training_images.append(image)  # Append the image
    label_dict[i] = filename  # Map the label to the filename

# Train the face recognizer
face_recognizer.train(training_images, np.array(labels))

# Function to match a reference face with the trained model
def match_face(reference_image, face_recognizer, label_dict):
    label, confidence = face_recognizer.predict(reference_image) # Predict the label and confidence
    return label_dict[label], confidence

# Comparing with reference image

In [32]:
reference_image_path = '/content/drive/MyDrive/aDatabase/Kamalesh.pgm'
reference_image = cv2.imread(reference_image_path, cv2.IMREAD_GRAYSCALE) # Load the reference image in grayscale

# Check if the reference image is loaded successfully
if reference_image is None:
    print(f'Could not load reference image from {reference_image_path}')
else:
    matched_image_name, confidence = match_face(reference_image, face_recognizer, label_dict)  # Match the reference face
    print(f'Matched Image: {matched_image_name} with confidence: {confidence}')  # Print the result
#less confidence value means more closely matched

Matched Image: me.pgm with confidence: 30.29846406670545
